In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.loc[:,['koi_period', 'koi_time0', 'koi_eccen', 'koi_longp', 'koi_impact', 'koi_ror']].head()
# df.head()

/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,koi_period,koi_time0,koi_eccen,koi_longp,koi_impact,koi_ror
0,54.418383,NaN,NaN,NaN,0.586,NaN
1,19.899140,NaN,NaN,NaN,0.969,NaN
2,1.736952,NaN,NaN,NaN,1.276,NaN
3,2.525592,NaN,NaN,NaN,0.701,NaN
4,4.134435,NaN,NaN,NaN,0.762,NaN


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df.loc[:,['koi_period', 'koi_impact', 'koi_teq','koi_insol']]
selected_features.head()
selected_features.shape

(6991, 4)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
y = df['koi_disposition']
y.shape

(6991,)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=42)
X_train.head()

,koi_period,koi_impact,koi_teq,koi_insol
6122,6.768901,0.150,1017,253.30
6370,0.733726,0.291,1867,2891.64
2879,7.652707,0.970,989,226.81
107,7.953547,0.300,696,55.37
29,4.959319,0.831,1103,349.40


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [9]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

classifier = LogisticRegression()

classifier.fit(X_train_scaled, y_train)
y_prediction = classifier.predict(X_test_scaled) 

print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.5761968338737364
Testing Data Score: 0.5589244851258581


/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [10]:
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [11]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50]}
grid = GridSearchCV(classifier, param_grid, verbose=3)

In [12]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] C=1 .............................................................
[CV] ................................. C=1, score=0.574, total=   0.0s
[CV] C=1 .............................................................
[CV] ................................. C=1, score=0.564, total=   0.0s
[CV] C=1 .............................................................
[CV] ................................. C=1, score=0.588, total=   0.0s
[CV] C=5 .............................................................
[CV] ................................. C=5, score=0.578, total=   0.0s
[CV] C=5 .............................................................
[CV] ................................. C=5, score=0.562, total=   0.0s
[CV] C=5 .............................................................
[CV] ................................. C=5, score=0.588, total=   0.0s
[CV] C=10 ............................................................


/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abb

[CV] ................................ C=10, score=0.577, total=   0.0s
[CV] C=10 ............................................................
[CV] ................................ C=10, score=0.561, total=   0.0s
[CV] C=10 ............................................................
[CV] ................................ C=10, score=0.586, total=   0.0s
[CV] C=50 ............................................................


/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/abby/miniconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/abb

[CV] ................................ C=50, score=0.579, total=   0.0s
[CV] C=50 ............................................................
[CV] ................................ C=50, score=0.561, total=   0.0s
[CV] C=50 ............................................................
[CV] ................................ C=50, score=0.584, total=   0.0s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None, param_grid={'C': [1, 5, 10, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [13]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 5}
0.5758153728781232


In [14]:
from sklearn.metrics import classification_report

grid_prediction = grid.predict(X_test_scaled)
print(classification_report(y_test, grid_prediction))

                precision    recall  f1-score   support

     CANDIDATE       0.31      0.02      0.04       411
     CONFIRMED       0.47      0.52      0.50       484
FALSE POSITIVE       0.60      0.84      0.70       853

      accuracy                           0.56      1748
     macro avg       0.46      0.46      0.41      1748
  weighted avg       0.50      0.56      0.49      1748



# Save the Model

In [15]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'LogistricRegression.sav'
joblib.dump(grid, filename)

['LogistricRegression.sav']